In [3]:
# Importing required libraries for optimization
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [ ]:
def least_squares(A, b):
    '''
    Solves the least squares problem Ax = b
    Args:
        A: co-efficients matrix
        b: constants vector
    Returns:
        x: solution vector
    '''

    return np.linalg.inv(A.T @ A) @ A.T @ b

In [ ]:
# Finite difference methods:
def forward_diff(f, x, h=1E-08):
    '''
    Solves the forward difference problem
    Args:
        f: function
        x: vector of variables
        h: step size
    Returns:
        delta_f: solution vector
    '''

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        delta_f[i] = (f(x_forw) - f(x)) / h # Calculate the forward difference

    return delta_f


def central_diff(f, x, h=1E-08):
    '''
    Solves the central difference problem
    Args:
        f: function
        x: vector of variables
        h: step size
    Returns:
        delta_f: solution vector
    '''

    delta_f = np.zeros(x.shape) # Initialize delta_f
    for i in range(x.shape[0]):
        x_forw = np.array(x) # Make a copy of x
        x_back = np.array(x) # Make a copy of x
        x_forw[i] += h   # Increment x_forw[i] by h
        x_back[i] -= h   # Decrement x_back[i] by h
        delta_f [i]= (f(x_forw) - f(x_back)) / (2*h) # Calculate the central difference

    return delta_f

In [ ]:
# Converting constrained optimization problem to unconstrained optimization problem:
# Using penalty method:
def constrained_to_unconstrained(f, con, x, p=100):
    '''
    Converts a constrained optimization problem to an unconstrained optimization problem
    Args:
        f: function
        con: constraint function
        x: vector of variables
        p: penalty parameter
    Returns:
        f_uncon: unconstrained function
    '''
    
    # Objective function
    def f_unconstrained(x):
        g, h = con(x) # Calculate the constraint function
        con_sum = 0 # Initialize the constraint sum
        for i in range(g.shape[0]):
            con_sum += max(0, g[i])**2 # Add the constraint sum
        for i in range(h.shape[0]):
            con_sum += h[i]**2  # Add the constraint sum
        return f(x) + p*con_sum # Return the unconstrained objective function
        
    return f_unconstrained

In [ ]:
# Unconstrained optimization:
# 1st order methods:

def steepest_descent(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the unconstrained problem using the steepest descent method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    convergence = False
    iter = 0
    while not convergence:
        s = -delta_f(f, x) # Calculate the search direction
        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
            
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new


def conjugate_gradient(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the unconstrained problem using the conjugate gradient method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    s = -delta_f(f, x) # Calculate the initial search direction
    s = s / np.linalg.norm(s) # Normalize the initial search direction
    alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the initial step size
    x_new = x + alpha*s # Calculate the initial x update

    convergence = False
    iter = 0
    while not convergence:
        if iter % x.shape[0] == 0:  # Check if the iteration counter is a multiple of the number of design variables
            s = -delta_f(f, x_new) # Calculate the search direction
        else:
            s = -delta_f(f, x_new) + (np.linalg.norm(delta_f(f, x_new))**2 / np.linalg.norm(delta_f(f, x))**2) * s # Calculate the search direction

        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x_new + alpha*s), 0, 1) # Calculate the step size
        x = x_new # Update x
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
        
    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new


# Hessian update rules:
def BFGS_update(B, delta_x, delta_delta):
    '''
    Calculates the BFGS update matrix
    Args:
        B: initial Hessian estimate
        delta_x: design variable update
        delta_delta: objective function update
    Returns:
        deta_B: update to the Hessian estimate
    '''

    return (1 + delta_delta.T@B@delta_delta / (delta_delta.T@delta_x)) * (delta_x@delta_x.T) / (delta_x.T@delta_delta) - \
        (delta_x@(delta_delta.T@B) + (delta_delta.T@B).T@delta_x.T) / (delta_x.T@delta_delta)

def DFP_update(B, delta_x, delta_delta):
    '''
    Calculates the DFP update matrix
    Args:
        B: initial Hessian estimate
        delta_x: design variable update
        delta_delta: objective function update
    Returns:
        deta_B: update to the Hessian estimate
    '''

    return delta_x@delta_x.T / (delta_x.T@delta_delta) - \
        (B@delta_delta)@(B@delta_delta).T / (delta_delta.T@B@delta_delta)


# Quasi-Newton method:
def quasi_newton(f, x, delta_f, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False, update_rule=BFGS_update):
    '''
    Optimizes the unconstrained problem using the quasi-Newton method
    Args:
        f: objective function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
        update_rule: update rule for the Hessian estimate
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''

    B = np.eye(x.shape[0]) # Initialize the Hessian approximation
    convergence = False
    iter = 0

    while not convergence:
        s = -B@delta_f(f, x)   # Calculate the search direction
        s = s / np.linalg.norm(s) # Normalize the search direction
        alpha = opt.fminbound(lambda alpha: f(x + alpha*s), 0, 1) # Calculate the step size
        x_new = x + alpha*s # Calculate the new x
        iter += 1 # Increment the iteration counter

         # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3

        B = B + update_rule(B, x_new - x, delta_f(f, x_new) - delta_f(f, x)) # Update the Hessian approximation
        x = x_new # Update x

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    else:
        return x_new

In [ ]:
# Constrained optimization methods:

def calc_lagrangian(f, con, x, lambda_, mu):
    '''
    Calculates the lagrangian of the function
    Args:
        f: objective function
        con: constraint function
        x: vector of variables
        lambda_: Lagrange equality multipliers
        mu: Lagrange inequality multipliers
    Returns:
        lagrangian: lagrangian of the function
    '''
    g, h = con(x)   # Calculate the inequality and equality constraints
    
    return f(x) + lambda_@h + mu@g   # Calculate the lagrangian


# Augmented Lagrangian method:
def calc_augmented_lagrangian(f, con, x, lambda_, mu, rho):
    '''
    Calculates the augmented Lagrangian of the function
    Args:
        f: objective function
        con: constraint function
        x: vector of variables
        lambda_: Lagrange equality multipliers
        mu: Lagrange inequality multipliers
        rho: augmented Lagrangian penalty parameter
    Returns:
        augmented_lagrangian: augmented Lagrangian of the function
    '''

    g, h = con(x) # Calculate the inequality and equality constraints
    con_sum = 0
    for i in range(g.shape[0]):
        con_sum += max(0, g[i])**2
    for i in range(h.shape[0]):
        con_sum += h[i]**2
    
    return calc_lagrangian(f, con, x, lambda_, mu) +  rho * con_sum # Calculate the augmented Lagrangian function


def augmented_lagrangian(f, con, x, delta_f, rho=1, grad_tol=1E-05, delta_f_tol=1E-05, delta_x_tol=1E-05, max_iter=100, full_output=False):
    '''
    Optimizes the constrained problem using the augmented Lagrangian method
    Args:
        f: objective function
        con: constraint function
        x: initial value of vector of variables
        delta_f: gradient of the objective function
        rho: augmented Lagrangian penalty parameter
        grad_tol: gradient tolerance
        delta_f_tol: objective function update tolerance
        delta_x_tol: design variable update tolerance
        max_iter: maximum number of iterations
        full_output: whether to return the full output or not
    Returns:
        x: solution vector
        f_val: objective function value
        exit_flag: exit flag
        iter: number of iterations
    '''
    
    g, h = con(x) # Calculate the initial constraint values
    mu0 = np.zeros(g.shape[0]).T # Initialize the inequality Lagrangian multipliers
    lambda_0 = np.zeros(h.shape[0]).T # Initialize the equality Lagrangian multipliers
    convergence = False
    iter = 0

    while not convergence:
        x_new = quasi_newton(lambda x: calc_augmented_lagrangian(f, con, x, lambda_0, mu0, rho), \
            x, delta_f, delta_f_tol, delta_x_tol, max_iter=100, full_output=False) # Optimize the augmented Lagrangian function
        g, h = con(x_new) # Calculate the constraint values
        mu_new = mu0 + rho*g # Calculate the new inequality Lagrangian multipliers
        # Check for negative inequality multipliers:
        for i in range(len(mu_new)):
            if mu_new[i] < 0:
                mu_new[i] = 0
        lambda_new = lambda_0 + rho*h # Calculate the new equality Lagrangian multipliers
        delta_L_new = delta_f(lambda x: calc_lagrangian(f, con, x, lambda_new, mu_new), x_new) # Calculate the new Lagrangian gradient
        iter += 1 # Increment the iteration counter
        
        # Check for convergence:
        if iter >= max_iter:    # Check if maximum number of iterations is reached
            convergence = True
            exit_flag = 0
        elif np.linalg.norm(delta_f(f, x_new)) <= grad_tol:    # Check if gradient is within tolerance
            convergence = True
            exit_flag = 1
        elif np.linalg.norm(x_new - x) <= delta_x_tol:  # Check if design variable update is within tolerance
            convergence = True
            exit_flag = 2
        elif np.linalg.norm(f(x_new) - f(x)) <= delta_f_tol:    # Check if objective function update is within tolerance
            convergence = True
            exit_flag = 3
        
        x = x_new # Update x
        mu0 = mu_new # Update the Lagrangian multipliers
        lambda_0 = lambda_new # Update the Lagrangian multipliers

    if full_output:
        return x_new, f(x_new), exit_flag, iter
    return x_new

In [ ]:
# Test objective function
def f(x):
    return -(x[0] + x[1])

# Test constraint function
def con(x):
    g = np.array([x[0]**2 + 2*x[1]**2 - 2])
    h = np.array([])
    return g, h

In [ ]:
x = np.array([[0, 0]], dtype=float).T # Initialize the design variable

f_unc = constrained_to_unconstrained(f, con, x) # Convert the constrained function to an unconstrained function

print(augmented_lagrangian(f, con, x, forward_diff, full_output=True)) # Optimize the constrained problem using the augmented Lagrangian method



In [4]:
# Scaling and descaling functions:
def scale(x_raw, x_ref):
    '''
    Scales the design variable
    Args:
        x_raw: raw design variable
        x_ref: reference design variable
    Returns:
        x_scaled: scaled design variable
    '''
    return np.divide(x_raw, x_ref)

def descale(x_scaled, x_ref):
    '''
    Descales the design variable
    Args:
        x_scaled: scaled design variable
        x_ref: reference design variable
    Returns:
        x_raw: raw design variable
    '''
    return np.multiply(x_scaled, x_ref)


In [8]:
def pressure_ratio_from_area_ratio(area_ratio):
    '''
    Calculates the pressure ratio from the area ratio
    Args:
        area_ratio: area ratio
    Returns:
        pressure_ratio: pressure ratio
    '''
    import constants as c

    # Temperory calculation variables:
    j = 2 * c.gamma / (c.gamma - 1)
    k = 2 / c.gamma
    l = (c.gamma - 1) / c.gamma

    pressure_ratio_0 = 0.017 # Initial guess for pressure ratio
    # Calculate the pressure ratio using fsolve:
    pressure_ratio = opt.fsolve(lambda pressure_ratio: area_ratio - c.Gamma / np.sqrt(j * pressure_ratio**k * (1 - pressure_ratio**l)), pressure_ratio_0, xtol=1E-06)
    return pressure_ratio


In [14]:
# Defining the objective function:
def objective(x_scaled):
    '''
    Calculates the objective function value
    Args:
        x_scaled: scaled design variable
    Returns:
        f_val: dry mass of rocket considering tank, chamber, injector plate and nozzle masses [kg]
    '''
    # Importing constants:
    import constants as c

    # Descaling the design variables:
    P_c, A_t, A_e = descale(x_scaled, c.x_ref)

    # Calculating fuel and oxidizer tank masses:
    # cylindrical section thickness [m]
    t_cyl = c.f * P_c * c.R_tank / c.sigma_tank
    t_sph = t_cyl / 2    # spherical section thickness [m]
    mass_UDMH_tank = (2*np.pi*t_cyl*c.R_tank*c.L_UDMH_tank + 4*np.pi*t_sph *
                        c.R_tank**2) * c.rho_tank    # mass of the UDMH tank [kg]
    mass_N2O4_tank = (2*np.pi*t_cyl*c.R_tank*c.L_N2O4_tank + 4*np.pi*t_sph *
                        c.R_tank**2) * c.rho_tank    # mass of the N2O4 tank [kg]

    # Ideal rocket theory calculations:
    mass_flow = c.Gamma * P_c * A_t / np.sqrt(c.R * c.T_c) # mass flow through the nozzle [kg/s]
    A_c = (mass_flow * c.R * c.T_c) / \
        (0.3 * P_c * np.sqrt(c.gamma * c.R * c.T_c))    # cross sectional area of the combustion chamber (assuming M = 0.3 for flow exiting nozzle) [m^2]
    R_c = np.sqrt(A_c / np.pi)    # radius of the combustion chamber [m]
    V_c = np.pi * R_c**2 * c.L_c    # volume of the combustion chamber [m^3]
    k_load = 1    # correction factor for high chamber pressures

    # Calculating the chamber and nozzle masses:
    mass_chamber = c.f * k_load * (2/(c.L_c/R_c) + 2) * c.rho/c.sigma * P_c * V_c    # mass of the combustion chamber [kg]
    mass_injector = c.f * (c.rho/c.sigma) * (1.2 * A_c * R_c * np.sqrt(P_c * c.sigma))  # mass of the injector plate [kg]
    mass_nozzle = c.f * (c.rho/c.sigma) * (A_t * ((A_e/A_t - 1)/np.sin(c.alpha)) * P_c * R_c)  # mass of the nozzle [kg]

    return (mass_UDMH_tank + mass_N2O4_tank + mass_chamber + mass_injector + mass_nozzle) / c.mass_dry_ref

In [12]:
# Defining constraints:
def constraints(x_scaled):
    '''
    Caclulates the constraint values
    Args:
        x_scaled: scaled design variable
    Returns:
        g: inequality constraint values
        h: equality constraint values
    '''

    # Importing constants:
    import constants as c
    
    # Descaling the design variables:
    P_c, A_t, A_e = descale(x_scaled, c.x_ref)

    mass_flow = c.Gamma * P_c * A_t / np.sqrt(c.R * c.T_c) # mass flow through the nozzle [kg/s]

    # Temperory calculation variables:
    j = 2 * c.gamma / (c.gamma - 1)
    k = 2 / c.gamma
    l = (c.gamma - 1) / c.gamma
    
    pressure_ratio = pressure_ratio_from_area_ratio(A_e/A_t) # nozzle pressure ratio
    P_e = P_c * pressure_ratio  # pressure at the exit of the nozzle [Pa]

    u_e = np.sqrt(j * c.R * c.T_c * (1 - pressure_ratio**l))    # velocity at the exit of the nozzle [m/s]
    thrust = mass_flow * u_e + (P_e - c.P_a) * A_e    # thrust [N]
    mass_wet = objective(x_scaled) * c.mass_dry_ref + c.mass_N2O4 + c.mass_UDMH    # wet mass of the rocket [kg]
    thrust_factor = thrust / (mass_wet * c.g0)  # thrust factor
    Isp = thrust / (mass_flow * c.g0)   # specific impulse [s]

    g = np.array([1 - Isp/c.Isp_ref])   # inequality constraint values
    h = np.array([1 - thrust_factor/c.thrust_factor_ref, 1 - mass_flow/c.mass_flow_ref])   # equality constraint values

    return g, h
